Ethics of AI Final Project

In [34]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import  DecisionTreeClassifier ,plot_tree, export_text# Decision tree algorithm and plotting functions for the Decision tree
from matplotlib import pyplot as plt # plotting/graphing


In [37]:
# fetch dataset and apply processing file
dataset = load_dataset("mstz/speeddating",trust_remote_code=True)["train"]

# convert to df
df = pd.DataFrame(dataset)


# split into x and y sets
X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date'])
X = pd.get_dummies(X)
y = df.is_match



In [38]:
# split into training and tests, ALL RANDOM STATES SET TO 0
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .33,random_state=0)

In [75]:
# Different models
# Random Forest

#Set estimators as 100, random state 0, samples of at least 2 in each leaf
clf = RandomForestClassifier(n_estimators = 125, random_state=0, min_samples_leaf=2)
clf.fit(x_train, y_train) # fit model

# get the roc auc and pring
cross_val_accuracy_roc_auc = round(cross_val_score(clf, x_train, y_train, cv = 10, scoring = 'roc_auc').mean()*100,2)

cross_val_accuracy_roc_auc

83.31

In [71]:
# Different models
# Decision Tree
clf = DecisionTreeClassifier(max_depth= 10, random_state=0, min_samples_leaf=5) # Set a max depth of 10, rnadom state of 0, min samples of 5 in each leaf
clf.fit(x_train, y_train) # fit model

#roc auc
cross_val_accuracy_roc_auc = (cross_val_score(clf, x_train, y_train, cv=10, scoring = 'roc_auc').mean()*100)
cross_val_accuracy_roc_auc

73.0374842942901